<a href="https://colab.research.google.com/github/shreedharmalpani/soc_check/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 2.5 MB 5.5 MB/s 
     |████████████████████████████████| 895 kB 33.8 MB/s 
     |████████████████████████████████| 3.3 MB 37.2 MB/s 


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt


In [3]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased',do_lower_case=True)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [4]:
tokens= tokenizer.encode('I absolutely hated this', return_tensors= 'pt')
tokens[0]

tensor([ 101,  178, 7284, 5687, 1142,  102])

In [5]:
result = model (tokens)
result.logits


tensor([[0.2039, 0.2030]], grad_fn=<AddmmBackward>)

In [6]:
int(torch.argmax(result.logits))

0

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
path="/content/drive/MyDrive/IMDB Dataset.csv"
df= pd.read_csv(path)
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [9]:
df.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [10]:
def delete_tags(text):
    str = ""
    f = True
    for char in text:
        char=char.lower()
        if char == '<':
            f = False

        if (f):
            str = str + char

        if char == '>':
            f == True
            str = str + ' '    
        
    return str

df['without'] = df['review'].apply(delete_tags)

In [11]:
df['label']= [ 1*(sent=='positive') for sent in df['sentiment'] ]

In [12]:
df


,review,sentiment,without,label
0,One of the other reviewers has mentioned that ...,positive,one of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,positive,a wonderful little production.,1
2,I thought this was a wonderful way to spend ti...,positive,i thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,negative,basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"petter mattei's ""love in the time of money"" is...",1
...,...,...,...,...
49995,I thought this movie did a down right good job...,positive,i thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,"bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,negative,i am a catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,negative,i'm going to have to disagree with the previou...,0


In [13]:

df["review"]= df["review"].apply(lambda x: " ".join(x.split()[:350]))
max_len = df["review"].apply(lambda x: len(x.split())).max()
print(max_len)

350


In [14]:
from transformers import BertTokenizer,AutoModel, BertModel
Max_Len= 350
tokenizer= BertTokenizer.from_pretrained("bert-base-uncased",do_lower_case=True)

class Final_embedding_dataset():
  
  def __init__(self,reviews,targets,tokenizer,max_len):
    
    self.reviews= reviews
    self.targets= targets
    self.tokenizer= tokenizer
    self.max_len= max_len
    
  def __len__(self):
    return len(self.reviews)

  def __getitem__(self,item):

    review= self.reviews[item]
    target= self.targets[item]

    encoding = self.tokenizer.encode_plus(review,add_special_tokens=True, padding="max_length",truncation=True, 
                                 return_tensors="pt",verbose=False,max_length=self.max_len,return_attention_mask=True)
    
# return_tensors="pt", gives us the embedding in form of tensors.
# tensors should be flattened.
    return {"review":review,
      "input_ids":encoding["input_ids"].flatten(),
      "attention_mask":encoding["attention_mask"].flatten(),
      "target": torch.tensor(target,dtype= torch.long)}

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device available for running: ")
print(device)
  

Device available for running: 
cpu


In [16]:
from sklearn.model_selection import train_test_split
df_train,df_test= train_test_split(df,test_size=0.2,random_state=1)
print(df_train.shape,df_test.shape)

(40000, 4) (10000, 4)


In [17]:
from torch.utils.data import DataLoader
def create_data_loader(df,batch_size,tokenizer,max_len):
  dataset= Final_embedding_dataset(df["review"].to_numpy(),df["sentiment"].to_numpy(),tokenizer,Max_Len)

  return DataLoader(dataset,batch_size=batch_size,num_workers=2)

Batch_Size= 16

train_data_loader= create_data_loader(df_train,Batch_Size,tokenizer,Max_Len)
test_data_loader= create_data_loader(df_test,Batch_Size,tokenizer,Max_Len)

In [18]:
class SentimentClassifier(nn.Module):
  def __init__(self,n_classes,model):
    super(SentimentClassifier,self).__init__()
    self.model = model
    self.dropout= nn.Dropout(p=0.3)
    self.output= nn.Linear(self.model.config.hidden_size,n_classes)

  def forward(self,input_ids,attention_mask):
    output= self.model(input_ids=input_ids,attention_mask=attention_mask)
    class_output= self.dropout(output["pooler_output"])

    return self.output(class_output)

In [19]:
from transformers.optimization import AdamW
from transformers import get_linear_schedule_with_warmup
n_classes= 2
classifier_model= SentimentClassifier(n_classes,model)
classifier_model= classifier_model.to(device)
EPOCHS= 10 
optimizer= AdamW(classifier_model.parameters(),lr=3e-5, correct_bias=False) 
scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps= len(train_data_loader)*EPOCHS)
loss_function= nn.CrossEntropyLoss().to(device)

In [20]:
def train_one_epoch(model,data_loader,loss_function,optimizer,device,scheduler,n_examples):

  model= model.train()

  losses=[]
  correct_predictions= 0

  for d in data_loader:
    input_ids= d.get("input_ids").to(device)
    attention_mask= d.get("attention_mask").to(device)
    targets= d.get("target").to(device)

    outputs= model(input_ids,attention_mask)
    
    _,preds= torch.max(outputs,dim=1)
    loss= loss_function(outputs,targets)

    correct_predictions+= torch.sum(preds==targets)
    losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(),max_norm=1.0) # is necessary to ensure that the gradients doesn't blow up
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    
    accuracy= correct_predictions/n_examples

  return accuracy,np.mean(losses)

In [21]:
def eval_model(model,data_loader,loss_function,device,n_examples):

  model= model.eval()

  losses=[]
  correct_predictions= 0

  with torch.no_grad():
    for d in data_loader:
      input_ids= d.get("input_ids").to(device)
      attention_mask= d.get("attention_mask").to(device)
      targets= d.get("target").to(device)
     
      outputs= model(input_ids,attention_mask)
    
      _,preds= torch.max(outputs,dim=1)
      loss= loss_function(outputs,targets)

      correct_predictions+= torch.sum(preds==targets)
      losses.append(loss.item())
    
      accuracy= correct_predictions.double()/n_examples

  return accuracy,np.mean(losses)

In [ ]:
from collections import defaultdict

#call model using classifier_model
# writing the training loop

history=defaultdict(list)
best_accuracy=0

for epoch in range(EPOCHS):
  print(f"EPOCH: {epoch+1}/{EPOCHS}")
  print("---"*10)

  train_acc,train_loss= train_one_epoch(classifier_model,train_data_loader,loss_function,optimizer,device,scheduler,len(df_train))
 
  print(f"Accuracy: {train_acc} Train loss: {train_loss}")
  
  test_acc, test_loss = eval_model(classifier_model,test_data_loader,loss_function,device, len(df_test) )

  print(f"test_accuracy: {test_acc} Test loss: {test_loss}")

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['test_acc'].append(test_acc)
  history['test_loss'].append(test_loss)


torch.save(model.state_dict(),"model.bin")

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
:def get_predictions(model,data_loader):

  model= model.eval()

  review_text= []
  predictions= []
  prediction_probs= []
  target_values= []

  with torch.no_grad():
    for d in data_loader:
      input_ids= d.get("input_ids").to(device)
      attention_mask= d.get("attention_mask").to(device)
      targets= d.get("target").to(device)

      outputs= model(input_ids,attention_mask)
    
      _,preds= torch.max(outputs,dim=1)
      probs= F.softmax(outputs,dim=1)

      review_text.append(d["review_text"]).cpu()
      predictions.append(preds).cpu()
      prediction_probs.append(probs).cpu()
      target_values.append(targets).cpu()
      

  return review_text, predictions, prediction_probs, target_values

y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(model,test_data_loader)
a,b= confusion_matrix(y_test,y_pred)
accuracy= (a[0]+b[0])/(a[0]+b[0]+a[1]+b[1])

In [26]:
accuracy*100

89.362